In [1]:
import pandas as pd

In [2]:
a = list()
with open('UCMR5_All.txt') as in_file:
    count = 0
    for row in in_file:
        print(row.strip())
        a.append(row)
        count += 1
        if count == 1:
            break

a

PWSID	PWSName	Size	FacilityID	FacilityName	FacilityWaterType	SamplePointID	SamplePointName	SamplePointType	AssociatedFacilityID	AssociatedSamplePointID	CollectionDate	SampleID	Contaminant	MRL	Units	MethodID	AnalyticalResultsSign	AnalyticalResultValue	SampleEventCode	MonitoringRequirement	Region	State	UCMR1SampleType


['PWSID\tPWSName\tSize\tFacilityID\tFacilityName\tFacilityWaterType\tSamplePointID\tSamplePointName\tSamplePointType\tAssociatedFacilityID\tAssociatedSamplePointID\tCollectionDate\tSampleID\tContaminant\tMRL\tUnits\tMethodID\tAnalyticalResultsSign\tAnalyticalResultValue\tSampleEventCode\tMonitoringRequirement\tRegion\tState\tUCMR1SampleType\n']

In [3]:
b = a[0].replace('\t',',').replace('\n', '')
cols = b.split(',')
print(cols)

['PWSID', 'PWSName', 'Size', 'FacilityID', 'FacilityName', 'FacilityWaterType', 'SamplePointID', 'SamplePointName', 'SamplePointType', 'AssociatedFacilityID', 'AssociatedSamplePointID', 'CollectionDate', 'SampleID', 'Contaminant', 'MRL', 'Units', 'MethodID', 'AnalyticalResultsSign', 'AnalyticalResultValue', 'SampleEventCode', 'MonitoringRequirement', 'Region', 'State', 'UCMR1SampleType']


In [4]:
out = {'PWSID': list(), 'Contaminant': list(), 'MRL': list(),
        'AnalyticalResultsSign': list(), 'AnalyticalResultValue': list(), 'Region': list(),
       'State': list()}

with open('UCMR5_All.txt') as in_file:
    count = 0
    for row in in_file:
        a = list()
        a.append(row)
        if count > 0:
            #b = a[0].replace('\t',',').replace('\n', '')
            vals = a[0].split('\t')
            out['PWSID'].append(vals[0])
            out['Contaminant'].append(vals[13])
            out['MRL'].append(vals[14])
            out['AnalyticalResultsSign'].append(vals[17])
            if vals[18] == '':
                out['AnalyticalResultValue'].append(float(0.0))
            else:
                out['AnalyticalResultValue'].append(float(vals[18]))
            out['Region'].append(vals[21])
            out['State'].append(vals[22])
        
        count += 1

        # if count > 10:
        #     break

#out

In [5]:
df = pd.DataFrame.from_dict(out)

In [6]:
df.head()

,PWSID,Contaminant,MRL,AnalyticalResultsSign,AnalyticalResultValue,Region,State
0,010106001,ADONA,0.003,<,0.0000,1,01
1,010106001,PFDA,0.003,<,0.0000,1,01
2,010106001,PFHxS,0.003,<,0.0000,1,01
3,010106001,PFBS,0.003,=,0.0036,1,01
4,010106001,NEtFOSAA,0.005,<,0.0000,1,01


In [7]:
a = df['State'].unique()
all_dict = {key: None for key in a}


In [8]:
for state in df['State'].unique():
    temp_df = pd.DataFrame(columns = df.columns) 
    temp_df = df[df['State'] == state]
    total_count = temp_df.shape[0]
    
    temp_df = temp_df[temp_df['AnalyticalResultsSign'] == '=']
    bad_count = temp_df.shape[0]
    
    temp_df = temp_df.reset_index()
    st_avgperc = 0

    for i in range(temp_df.shape[0]):
        minval = temp_df['MRL'][i]
        repval = temp_df['AnalyticalResultValue'][i]
        st_avgperc += (repval - float(minval))/float(minval)

    if temp_df.shape[0] == 0:
        stuff = 0
    else:
        stuff = round(st_avgperc/temp_df.shape[0], 4)

    all_dict[state] = {'Percentage': stuff, 'Sample Ratio': round(bad_count/total_count, 4)}

In [9]:
all_dict

{'01': {'Percentage': 1.3934, 'Sample Ratio': 0.1056},
 '02': {'Percentage': 0, 'Sample Ratio': 0.0},
 '04': {'Percentage': 0, 'Sample Ratio': 0.0},
 '05': {'Percentage': 3.0074, 'Sample Ratio': 0.0091},
 '06': {'Percentage': 8.6186, 'Sample Ratio': 0.0313},
 '08': {'Percentage': 5.106, 'Sample Ratio': 0.0269},
 '09': {'Percentage': 5.0423, 'Sample Ratio': 0.0285},
 '10': {'Percentage': 0, 'Sample Ratio': 0.0},
 'AK': {'Percentage': 0.6846, 'Sample Ratio': 0.0066},
 'AL': {'Percentage': 2.0845, 'Sample Ratio': 0.027},
 'AR': {'Percentage': 1.4074, 'Sample Ratio': 0.0073},
 'AS': {'Percentage': 0.2564, 'Sample Ratio': 0.0022},
 'AZ': {'Percentage': 5.8651, 'Sample Ratio': 0.0465},
 'CA': {'Percentage': 1.7066, 'Sample Ratio': 0.0219},
 'CO': {'Percentage': 2.0982, 'Sample Ratio': 0.0301},
 'CT': {'Percentage': 0.532, 'Sample Ratio': 0.0333},
 'DC': {'Percentage': 0.4439, 'Sample Ratio': 0.0253},
 'DE': {'Percentage': 2.3635, 'Sample Ratio': 0.0604},
 'FL': {'Percentage': 1.4765, 'Sample

In [10]:
max_states = ['a','b','c','d','e']
ignore = ['01','02','03','04','05','06','07','08','09','10', 'VI','NN','GU']

for i in range(len(max_states)):
    high = 0
    loc = 'asdf'
    for key in all_dict.keys():
        val = all_dict[key]['Percentage']
        if (val > high) and (key not in max_states) and (key not in ignore):
            high = val
            loc = key
            
    max_states[i] = loc

max_states

['AZ', 'IA', 'SD', 'MN', 'NM']

In [11]:
high_contan_states = ['Arizona','Iowa','South Dakota','Minnesota', 'New Mexico']

In [12]:
google = ['North Carolina', 'South Carolina', 'Alabama', 'Colorado', 'Ohio', 'New Jersey', 'Massachusetts', 'Rhode Island', 'Minnesota',
          'Wisconsin','Florida']